In [176]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.listdir('../input/chess-pieces-detection-images-dataset')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

['Rook-resize',
 'pawn_resized',
 'knight-resize',
 'Queen-Resized',
 'bishop_resized']

In [177]:
Rooks = '../input/chess-pieces-detection-images-dataset/Rook-resize'
Pawns = '../input/chess-pieces-detection-images-dataset/pawn_resized'
Knights = '../input/chess-pieces-detection-images-dataset/knight-resize'
Queens = '../input/chess-pieces-detection-images-dataset/Queen-Resized'
Bishops = '../input/chess-pieces-detection-images-dataset/bishop_resized'
classes = [Rooks, Pawns, Knights, Queens, Bishops]

In [178]:
import cv2
"""
def feature_builder(clss):
    lst = []
    for img in os.listdir(clss):
        f = cv2.imread(os.path.join(clss,img))
        f = cv2.cvtColor(f , cv2.COLOR_BGR2GRAY)        #convert colored images into grayscale format
        f = cv2.resize(f , (50,50))
        f = f / 255.0 #potential depricatipn candidate
        f = np.expand_dims(f, axis=-1)
        lst.append(f)
    return lst"""
def feature_builder(clss):
    lst = []
    for img in os.listdir(clss):
        f = cv2.imread(os.path.join(clss, img), cv2.IMREAD_GRAYSCALE)  # Read image in grayscale directly
        f = cv2.resize(f, (100, 100))
        f = f / 255.0  # Normalize the image data to 0-1
        f = np.expand_dims(f, axis=-1)  # Add channel dimension
        lst.append(f)
    return lst

In [179]:
features = []
for c in classes:
    features += feature_builder(c)


    

In [180]:
X = np.array(features)#convert to compatable numpy array
X = X.reshape(-1, 100, 100, 1)

In [181]:
#Label extraction
labels = []
count = 0
for c in classes:
    for img in os.listdir(c):
        labels.append(count)
    count += 1
    

In [182]:
Y = np.array(labels)

In [183]:
#Process x and y lists
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical, normalize

#X = X.reshape(651,2500) #flatten the input for the model
#X = normalize(X) #make sure that x has values between 0 and 1
Y = to_categorical(Y)
print(Y)

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [184]:
from sklearn.model_selection import train_test_split
#train test split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,train_size=0.8)
X_train.shape

(520, 100, 100, 1)

In [185]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5))

In [186]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, epochs=10, 
                    validation_data=(X_test, Y_test))

Epoch 1/10
17/17 [==============================] - 4s 208ms/step - loss: 1.6958 - accuracy: 0.2135 - val_loss: 1.5965 - val_accuracy: 0.3053
Epoch 2/10
17/17 [==============================] - 3s 175ms/step - loss: 1.5549 - accuracy: 0.3077 - val_loss: 1.5335 - val_accuracy: 0.3435
Epoch 3/10
17/17 [==============================] - 3s 174ms/step - loss: 1.4358 - accuracy: 0.4000 - val_loss: 1.4290 - val_accuracy: 0.4275
Epoch 4/10
17/17 [==============================] - 3s 181ms/step - loss: 1.2215 - accuracy: 0.5269 - val_loss: 1.4796 - val_accuracy: 0.4122
Epoch 5/10
17/17 [==============================] - 3s 178ms/step - loss: 1.0019 - accuracy: 0.6019 - val_loss: 1.5079 - val_accuracy: 0.4580
Epoch 6/10
17/17 [==============================] - 3s 173ms/step - loss: 0.8055 - accuracy: 0.6942 - val_loss: 1.5303 - val_accuracy: 0.4580
Epoch 7/10
17/17 [==============================] - 3s 176ms/step - loss: 0.6608 - accuracy: 0.7577 - val_loss: 1.5708 - val_accuracy: 0.4962
Epoch 